In [1]:
!pip install onnx

import time
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 39.8 MB/s eta 0:00:00


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train_flat = x_train.reshape(-1, 784)
x_test_flat = x_test.reshape(-1, 784)

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
tf_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

tf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()
tf_model.fit(x_train, y_train_cat, epochs=5, batch_size=32, verbose=2)
tf_training_time = time.time() - start_time
print(f"TensorFlow Training Time: {tf_training_time:.2f} seconds")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 - 14s - 7ms/step - accuracy: 0.9137 - loss: 0.3046
Epoch 2/5
1875/1875 - 12s - 6ms/step - accuracy: 0.9562 - loss: 0.1488
Epoch 3/5
1875/1875 - 14s - 7ms/step - accuracy: 0.9676 - loss: 0.1081
Epoch 4/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9747 - loss: 0.0843
Epoch 5/5
1875/1875 - 9s - 5ms/step - accuracy: 0.9788 - loss: 0.0691
TensorFlow Training Time: 55.07 seconds


In [4]:
start_time = time.time()
loss, acc = tf_model.evaluate(x_test, y_test_cat, verbose=0)
tf_inference_time = time.time() - start_time
print(f"TensorFlow Test Accuracy: {acc:.4f}")
print(f"TensorFlow Inference Time: {tf_inference_time:.4f} seconds")


TensorFlow Test Accuracy: 0.9706
TensorFlow Inference Time: 1.1286 seconds


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()
with open("/content/drive/MyDrive/Colab Notebooks/EM Report/model.tflite", "wb") as f:
    f.write(tflite_model)
print("TensorFlow model converted to model.tflite")

Saved artifact at '/tmp/tmp83j9hubc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135462624121232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135462624122192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135462624121616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135462624122000: TensorSpec(shape=(), dtype=tf.resource, name=None)
TensorFlow model converted to model.tflite


In [7]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/EM Report', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='/content/drive/MyDrive/Colab Notebooks/EM Report', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pt_model = PyTorchModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pt_model.parameters(), lr=0.001)


In [9]:
start_time = time.time()
pt_model.train()
for epoch in range(5):
    running_loss = 0.0
    correct = 0
    total = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = pt_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{5}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

pt_training_time = time.time() - start_time

print(f"PyTorch Training Time: {pt_training_time:.2f} seconds")

Epoch 1/5, Loss: 0.3309, Accuracy: 0.9101
Epoch 2/5, Loss: 0.1608, Accuracy: 0.9532
Epoch 3/5, Loss: 0.1177, Accuracy: 0.9655
Epoch 4/5, Loss: 0.0920, Accuracy: 0.9724
Epoch 5/5, Loss: 0.0754, Accuracy: 0.9773
PyTorch Training Time: 88.54 seconds


In [10]:
pt_model.eval()
correct = 0
total = 0
start_time = time.time()
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = pt_model(data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)
pt_inference_time = time.time() - start_time
print(f"PyTorch Test Accuracy: {correct / total:.4f}")
print(f"PyTorch Inference Time: {pt_inference_time:.4f} seconds")


PyTorch Test Accuracy: 0.9712
PyTorch Inference Time: 2.1251 seconds


In [11]:
dummy_input = torch.randn(1, 784).to(device)
torch.onnx.export(pt_model, dummy_input, "/content/drive/MyDrive/Colab Notebooks/EM Report/model.onnx", input_names=["input"], output_names=["output"])
print("PyTorch model converted to model.onnx")

PyTorch model converted to model.onnx
